In [1]:
import numpy as np
import pandas as pd
from typing import Optional, Tuple

In [2]:
#Posiciones
POSICION_FUENTES = {'F1':(289782,6363955,1.2),
                    'F2':(289788,6363955,1.2),
                    'F3':(289782,6363979,1.2),
                    'FC':(289782,6363967,1.2)}
POSICION_RECEPTORES = {'R1':(289827,6363825,1.5),
                       'R2':(289398,6363754,1.5),
                       'R3':(290587,6364355,1.5),
                       'R4':(290736,6363776,1.5),
                       'R5':(289782,6364017,1.5)}

In [3]:
LW_FUENTES = {'F1':(65.1,78.3,87.6,96.4,99.8,99.1,94.3,84.4),
              'F2':(65.1,78.3,87.6,96.4,99.8,99.1,94.3,84.4),
              'F3':(53.1,66.2,75.7,84.1,87.3,87.5,82.3,72.2),
              'FC':(68.25,81.44,90.75,99.54,102.93,102.26,97.45,87.54)}

In [4]:
FREQ_OCT = [63,125,250,500,1000,2000,4000,8000]
FREQ_TER = [50,63,80,100,125,160,200,250,315,400,500,630,800,1000,1250,1600,2000,2500,3150,4000,5000,6300,8000,10000]
C = 343.3

In [5]:
coeficiente_atm = pd.read_csv("coeficientes temperatura.txt", delimiter=" ")

In [6]:
valores_humedad = coeficiente_atm['%'].values
valores_temperatura = coeficiente_atm['°C'].values

In [7]:
_lambda = C/np.array(FREQ_OCT)

In [9]:
def distancia(posicion1:Tuple[float], posicion2:Tuple[float]):
    posicion1 = np.array(posicion1)
    posicion2 = np.array(posicion2)
    return np.sqrt(((posicion1-posicion2)**2).sum())

In [10]:
distancia(POSICION_FUENTES['FC'],POSICION_RECEPTORES['R2'])

439.1185375271693

In [11]:
def atenuacion_div(distancia, d0:int=1):
    return 20*np.log10(distancia/d0)+11

In [12]:
def alpha_atm(temperatura:int, humedad:int):
    if humedad not in valores_humedad:
        return f'el valor {humedad} no se encuentra en la tabla'
    if temperatura not in valores_temperatura:
        return f'el valor {temperatura} no se encuentra en la tabla'
    return coeficiente_atm.loc[(coeficiente_atm['%']==humedad) &
                                (coeficiente_atm['°C']==temperatura)].drop(columns=['%', '°C'])

In [13]:
def atenuacion_atm(distancia:float, alpha):
    return alpha*distancia/1000

In [14]:
class Source:
    def __init__(self, posicion:Tuple[float, float, float], lw:float) -> None:
        self.posicion = posicion
        self.lw = lw
        self.altura = posicion[-1]

    @property
    def get_pos(self):
        return self.posicion[0], self.posicion[2]

In [15]:
class Receiver:
    def __init__(self, posicion:Tuple[float, float, float]) -> None:
        self.posicion = posicion
        self.altura = posicion[-1]

    @property
    def get_pos(self):
        return self.posicion[0], self.posicion[2]
    

In [16]:
class AtenuacionPiso:
    def __init__(self, h:float, g:float, dp:float) -> None:
        self.h = h
        self.g = g
        self.dp = dp
        self.__calculate_functions__()

    def __calculate_functions__(self):
        self.a = 1.5 + 3*np.exp(-0.12*(self.h-5)**2)*(1-np.exp(-self.dp/50))+5.7*np.exp(-0.09*self.h**2)*(1-np.exp(-2.8*10**(-6*self.dp**2)))
        self.b = 1.5 + 8.6*np.exp(-0.09*self.h**2)*(1-np.exp(-self.dp/50))
        self.c = 1.5 + 14*np.exp(-0.46*self.h**2)*(1-np.exp(-self.dp/50))
        self.d = 1.5 + 5*np.exp(-0.9*self.h**2)*(1-np.exp(-self.dp/50))

    def a_octava(self):
        a_63 = 1.5
        a_125 = -1.5+self.g*self.a
        a_250 = -1.5+self.g*self.b
        a_500 = -1.5+self.g*self.c
        a_1000 = -1.5+self.g*self.d
        a_2000 = -1.5*(1-self.g)
        a_4000 = -1.5*(1-self.g)
        a_8000 = -1.5*(1-self.g)
        return np.array([a_63, a_125, a_250, a_500, a_1000, a_2000, a_4000, a_8000])
    

In [17]:
class AGround:
    def __init__(self, g_source:float, g_middle:float, g_receiver:float, source:Source, receiver:Receiver) -> None:
        self.g_source = g_source
        self.g_middle = g_middle
        self.g_receiver = g_receiver
        self.source = source
        self.receiver = receiver 
        
    @property
    def source_region(self):
        return 30*self.source.altura
    
    @property
    def receiver_region(self):
        return 30*self.receiver.altura
    
    @property
    def dp(self):
        return distancia(self.source.posicion[:2],self.receiver.posicion[:2])
    
    @property
    def middle_region(self):
        result = self.dp-self.source_region-self.receiver_region
        return result if result>0 else None
    
    def a_s(self):
        calculos_as = AtenuacionPiso(self.source.altura, self.g_source, self.dp)
        return calculos_as.a_octava()
    def a_r(self):
        calculos_ar = AtenuacionPiso(self.receiver.altura, self.g_receiver, self.dp)
        return calculos_ar.a_octava()
    def a_m(self):
        q = 1 - 30*(self.source.altura+self.receiver.altura)/self.dp if self.middle_region else 0
        return np.array([-3*q**2,
                         -3*q*1-self.g_middle,
                         -3*q*1-self.g_middle,
                         -3*q*1-self.g_middle,
                         -3*q*1-self.g_middle,
                         -3*q*1-self.g_middle,
                         -3*q*1-self.g_middle,
                         -3*q*1-self.g_middle])
    
    @property
    def get_atenuacion(self):
        return self.a_s() + self.a_r() + self.a_m()

In [18]:
class Barrera:
    def __init__(self, x:float, altura:float, espesor:Optional[float]=0) -> None:
        self.x = x
        self.altura = altura
        self.espesor = espesor
    
    @property
    def get_pos(self):
        return self.x, self.altura
    
    @property
    def get_delta_pos(self):
        return self.x + self.espesor, self.altura
    

In [19]:
class AtenuacionBarrera:
    def __init__(self, barrera:Barrera, receiver:Receiver, source:Source) -> None:
        self.receiver = receiver
        self.source = source
        self.barrera = barrera
        self.e = barrera.espesor
        self.c2 = 20

    @property
    def dss(self):
        return distancia(self.source.get_pos, self.barrera.get_pos)
    
    @property
    def dsr(self):
        return distancia(self.barrera.get_delta_pos, self.receiver.get_pos)
    
    @property
    def d(self):
        return distancia(self.receiver.get_pos, self.source.get_pos)
    
    @property
    def a(self):
        return np.abs(self.receiver.altura - self.source.altura)
    
    @property
    def z(self):
        return np.sqrt((self.dss+self.dsr+self.e)**2+(self.a)**2)-self.d
    
    @property
    def kmet(self):
        return np.exp(-(1/2000)*np.sqrt(
            self.dss*self.dsr*self.d/(2*self.z))) if self.z>0 else 1
    
    def _c3_(self, _lambda): 
        return ((1+(5*_lambda/self.e)**2)/(1/3+(5*_lambda/self.e)**2)) if self.e!=0 else 1
    
    def _dz_(self, _lambda):
        return 10*np.log10(3+(self.c2/_lambda)*self._c3_(_lambda)*self.z*self.kmet)
    

In [20]:
class AtenuacionMain:
    def __init__(self, receiver:Receiver, source:Source, barrera:Optional[Barrera]=None) -> None:
        self.receiver = receiver
        self.source = source
        self.barrera = barrera
    
    

In [21]:
barrera = Barrera(30, 3)
receptor1 = Receiver(POSICION_RECEPTORES['R1'])
fuente = Source(POSICION_FUENTES['FC'], LW_FUENTES['FC'])

In [24]:
abar = AtenuacionBarrera(barrera, receptor1, fuente)

<bound method AtenuacionBarrera._dz_ of <__main__.AtenuacionBarrera object at 0x000001F41F885E10>>

In [ ]:
a = AGround(0, 1, 0.5,fuente1, receptor1 )
a.get_atenuacion

array([ 3.        , -0.64244502,  2.6407284 ,  1.57790975, -0.65789601,
       -1.        , -1.        , -1.        ])

In [ ]:
a.e

0.015

In [ ]:
a._dz_(_lambda)

array([ 6.23082551,  7.3060101 ,  8.89610994, 10.97296653, 13.42876341,
       16.13332159, 18.98451202, 21.92208322])

In [ ]:
barrera.get_pos

(30, 3)